In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle


import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD   
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words(G).pkl','wb'))
pickle.dump(classes,open('classes(G).pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
#training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('chatbot_model(G).h5', hist)

print("model created")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


253 documents
74 classes ['alcohol_and_liver_health', 'cirrhosis_symptoms', 'fatty_liver_disease', 'hepatitis_symptoms', 'liver_disease_alcohol_abuse', 'liver_disease_alcohol_impact', 'liver_disease_and_antioxidants', 'liver_disease_and_bile_problems', 'liver_disease_and_blood_clotting', 'liver_disease_and_caffeine', 'liver_disease_and_diabetes', 'liver_disease_and_fatigue', 'liver_disease_and_herbal_remedies', 'liver_disease_and_herbal_supplements', 'liver_disease_and_inflammation', 'liver_disease_and_kidney_damage', 'liver_disease_and_mental_health', 'liver_disease_and_pregnancy', 'liver_disease_and_skin_rash', 'liver_disease_and_stress', 'liver_disease_and_urine_changes', 'liver_disease_and_vitamins', 'liver_disease_and_weight_loss', 'liver_disease_autoimmune', 'liver_disease_blood_pressure', 'liver_disease_blood_tests', 'liver_disease_cancer', 'liver_disease_cause', 'liver_disease_cirrhosis', 'liver_disease_diagnosis', 'liver_disease_diagnosis_method', 'liver_disease_diet_avoid', '

C:\Users\abhi\anaconda3\envs\env\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


51/51 [==============================] - 1s 2ms/step - loss: 4.3317 - accuracy: 0.0198
Epoch 2/300
51/51 [==============================] - 0s 2ms/step - loss: 4.2823 - accuracy: 0.0277
Epoch 3/300
51/51 [==============================] - 0s 2ms/step - loss: 4.2477 - accuracy: 0.0277
Epoch 4/300
51/51 [==============================] - 0s 3ms/step - loss: 4.2062 - accuracy: 0.0395
Epoch 5/300
51/51 [==============================] - 0s 3ms/step - loss: 4.1453 - accuracy: 0.0593
Epoch 6/300
51/51 [==============================] - 0s 3ms/step - loss: 4.0935 - accuracy: 0.0514
Epoch 7/300
51/51 [==============================] - 0s 3ms/step - loss: 4.0329 - accuracy: 0.0791
Epoch 8/300
51/51 [==============================] - 0s 3ms/step - loss: 3.9262 - accuracy: 0.1107
Epoch 9/300
51/51 [==============================] - 0s 3ms/step - loss: 3.8882 - accuracy: 0.1067
Epoch 10/300
51/51 [==============================] - 0s 4ms/step - loss: 3.6876 - accuracy: 0.1581
Epoch 11/300
51/51 [=

In [2]:
# !pip freeze > requirements.txt

In [3]:
# !python --version

In [4]:
#! pip install -r requirements.txt
# pip install --upgrade googletrans==4.0.0-rc1
# pip install translate